In [1]:
from datetime import datetime
import MetaTrader5 as mt5
#import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#connect to mt5
if not mt5.initialize():
   print("Unable to connect to MT5")
   mt5.shutdown()

In [3]:
symbol_info = mt5.symbol_info("EURUSD")
print("EurUsd: ")
print("Spread: ", symbol_info.spread, " Digits: ", symbol_info.digits)
print("Bid: ", symbol_info.bid, " Ask: ", symbol_info.ask)

EurUsd: 
Spread:  2  Digits:  5
Bid:  1.21856  Ask:  1.21858


In [4]:
date = datetime.now().strftime("%Y %m %d %H %M").split()
date[3] = (int(date[3]) + 3) % 24
for num, d in enumerate(date):
   date[num] = int(d)

eurusd_rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M5, datetime(date[0],date[1],date[2],date[3],date[4]), 10080)
# for eurusd in eurusd_rates:
#    print(datetime.fromtimestamp(eurusd[0]))
print(type(eurusd_rates))
print(eurusd_rates)

<class 'numpy.ndarray'>
[(1606897200, 1.20766, 1.20769, 1.20754, 1.20767, 122, 1, 0)
 (1606897500, 1.20764, 1.20772, 1.20754, 1.20765,  80, 1, 0)
 (1606897800, 1.20765, 1.20769, 1.20729, 1.2073 ,  97, 1, 0) ...
 (1611322500, 1.21818, 1.21835, 1.2181 , 1.21813, 160, 0, 0)
 (1611322800, 1.21814, 1.21834, 1.21807, 1.21818, 199, 0, 0)
 (1611323100, 1.21817, 1.21872, 1.21816, 1.21856, 222, 0, 0)]


In [5]:
test = np.array(eurusd_rates, dtype=[('timestamp', 'int64'), ("open", "float64"), ("high", "float64"),
    ("low", "float64"), ("close", "float64"), ("volume", "int64"), ("spread", "int64"), ("real_volume", "int64")])
print("Spread:")
print("moy: ", test["spread"].mean(), " std: ", test["spread"].std(), " min: ", test["spread"].min(), " max: ", test["spread"].max(), " per25: ", np.percentile(test["spread"], 25), " median: ", np.percentile(test["spread"], 50), " per75: ", np.percentile(test["spread"], 75))

Spread:
moy:  0.8967261904761905  std:  3.4008269880730664  min:  0  max:  171  per25:  0.0  median:  0.0  per75:  1.0


In [6]:
import talib
import pandas as pd

rsi = talib.RSI(test["close"], timeperiod=14)
print(len(rsi))
print(rsi[:20])

10080
[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan         nan 59.61002786 59.07836059 63.20196248 58.82461821
 54.58373022 38.79907687]


In [7]:
eurusd_M5 = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 99_999)
eurusd_M30 = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M30, 0, 99_999)

print(eurusd_M5[-1])
print(eurusd_M30[-1])
#mt5.last_error()
pd_eurusd_M5 = pd.DataFrame(eurusd_M5)
pd_eurusd_M30 = pd.DataFrame(eurusd_M30)


cci5 = talib.CCI(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod= 14)

print("CCI  M5: ", cci5.iloc[-1])

(1611323100, 1.21817, 1.21872, 1.21816, 1.21856, 222, 0, 0)
(1611322200, 1.21807, 1.21872, 1.2179, 1.21856, 798, 0, 0)
CCI  M5:  139.8943273529703


In [8]:
# Bollinger Band
pd_eurusd_M5["upperBBands_M5"], pd_eurusd_M5["middleBBands_M5"], pd_eurusd_M5["lowerBBands_M5"] = talib.BBANDS(pd_eurusd_M5["close"], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

# Double Exponential Moving Average
pd_eurusd_M5["DEMA_M5"] = talib.DEMA(pd_eurusd_M5["close"], timeperiod= 21)

# MidPoint over period
pd_eurusd_M5["Midpoint_M5"] = talib.MIDPOINT(pd_eurusd_M5["close"], timeperiod= 14)

# Midpoint Price over period
pd_eurusd_M5["Midprice_M5"] = talib.MIDPRICE(pd_eurusd_M5["high"], pd_eurusd_M5["low"], timeperiod= 14)

# Parabolic SAR
pd_eurusd_M5["SAR_M5"] = talib.SAR(pd_eurusd_M5["high"], pd_eurusd_M5["low"], acceleration=0.02, maximum=0.2)

# Aroon Oscillator
pd_eurusd_M5["Aroon_M5"] = talib.AROONOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], timeperiod=14)

# Balance Of Power
pd_eurusd_M5["BOP_M5"] = talib.BOP(pd_eurusd_M5["open"], pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"])

# CCI
pd_eurusd_M5["CCI_M5"] = talib.CCI(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod= 14)

# MACD
pd_eurusd_M5["MACD_M5"], pd_eurusd_M5["MACDSignal_M5"], pd_eurusd_M5["MACDHist_M5"] = talib.MACD(pd_eurusd_M5["close"], fastperiod=12, slowperiod=26, signalperiod=9)

# Momentum
pd_eurusd_M5["Mom_M5"] = talib.MOM(pd_eurusd_M5["close"], timeperiod=10)

# Percentage Price Oscillator
pd_eurusd_M5["PPO_M5"] = talib.PPO(pd_eurusd_M5["close"], fastperiod=12, slowperiod=26, matype=0)

# Rate of change
pd_eurusd_M5["ROC_M5"] = talib.ROC(pd_eurusd_M5["close"], timeperiod=10)

# Relative Strength Index
pd_eurusd_M5["RSI_M5"] = talib.RSI(pd_eurusd_M5["close"], timeperiod=14)

# Stochastic
pd_eurusd_M5["Stoc_slowk_M5"], pd_eurusd_M5["Stoc_slowd_M5"] = talib.STOCH(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# TRIX
pd_eurusd_M5["TRIX_M5"] = talib.TRIX(pd_eurusd_M5["close"], timeperiod=30)

# Ultimate Oscillator
pd_eurusd_M5["UltOsc_M5"] = talib.ULTOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

# Williams' %R
pd_eurusd_M5["WillamsR_M5"] = talib.WILLR(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], timeperiod=14)

# Chaikin A/D Oscillator
pd_eurusd_M5["ADOsc_M5"] = talib.ADOSC(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"], pd_eurusd_M5["tick_volume"], fastperiod=3, slowperiod=10)

# On Balance Volume
pd_eurusd_M5["OBV_M5"] = talib.OBV(pd_eurusd_M5["close"], pd_eurusd_M5["tick_volume"])

# True Range
pd_eurusd_M5["TrueRange_M5"] = talib.TRANGE(pd_eurusd_M5["high"], pd_eurusd_M5["low"], pd_eurusd_M5["close"])

In [9]:
#Merge M5 and M30 Dataframes
pd_eurusd = pd_eurusd_M5.merge(pd_eurusd_M30, how="left", left_on= "time", right_on= "time")

print(pd_eurusd[:40])

          time   open_x   high_x    low_x  close_x  tick_volume_x  spread_x  \
0   1568767500  1.10711  1.10717  1.10704  1.10710             25         7   
1   1568767800  1.10710  1.10711  1.10709  1.10709             26         9   
2   1568768100  1.10709  1.10716  1.10704  1.10711             62         6   
3   1568768400  1.10711  1.10731  1.10706  1.10726             65         4   
4   1568768700  1.10726  1.10727  1.10717  1.10726             30         3   
5   1568769000  1.10726  1.10726  1.10720  1.10720             31         3   
6   1568769300  1.10720  1.10726  1.10715  1.10725             40         3   
7   1568769600  1.10725  1.10732  1.10721  1.10727             39         3   
8   1568769900  1.10727  1.10727  1.10716  1.10719             47         3   
9   1568770200  1.10719  1.10725  1.10716  1.10716             55         3   
10  1568770500  1.10716  1.10725  1.10705  1.10707             55         4   
11  1568770800  1.10707  1.10711  1.10705  1.10710  

In [10]:
# Remove first empty lines of M30

for i in range(0,7):
    if pd_eurusd["time"].iloc[0] % 1800 == 0:
        break
    else:
        pd_eurusd = pd_eurusd.drop([i])

pd_eurusd[0:12]

,time,open_x,high_x,low_x,close_x,tick_volume_x,spread_x,real_volume_x,upperBBands_M5,middleBBands_M5,...,ADOsc_M5,OBV_M5,TrueRange_M5,open_y,high_y,low_y,close_y,tick_volume_y,spread_y,real_volume_y
3,1568768400,1.10711,1.10731,1.10706,1.10726,65,4,0,NaN,NaN,...,NaN,126.0,0.00025,1.10711,1.10732,1.10706,1.10719,252.0,3.0,0.0
4,1568768700,1.10726,1.10727,1.10717,1.10726,30,3,0,NaN,NaN,...,NaN,126.0,0.00010,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1568769000,1.10726,1.10726,1.10720,1.10720,31,3,0,NaN,NaN,...,NaN,95.0,0.00006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1568769300,1.10720,1.10726,1.10715,1.10725,40,3,0,NaN,NaN,...,NaN,135.0,0.00011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1568769600,1.10725,1.10732,1.10721,1.10727,39,3,0,NaN,NaN,...,NaN,174.0,0.00011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1568769900,1.10727,1.10727,1.10716,1.10719,47,3,0,NaN,NaN,...,NaN,127.0,0.00011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1568770200,1.10719,1.10725,1.10716,1.10716,55,3,0,NaN,NaN,...,-8.832760,72.0,0.00009,1.10719,1.10725,1.10705,1.10710,262.0,2.0,0.0
10,1568770500,1.10716,1.10725,1.10705,1.10707,55,4,0,NaN,NaN,...,-30.979634,17.0,0.00020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1568770800,1.10707,1.10711,1.10705,1.10710,33,2,0,NaN,NaN,...,-30.223388,50.0,0.00006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,1568771100,1.10710,1.10715,1.10705,1.10712,55,2,0,NaN,NaN,...,-20.166434,105.0,0.00010,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Adjust M30 open, high, low, close values

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 0:
        pd_eurusd["open_y"].iloc[i] = pd_eurusd["open_x"].iloc[i]
        pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_x"].iloc[i]
        pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_x"].iloc[i]
        pd_eurusd["close_y"].iloc[i] = pd_eurusd["close_x"].iloc[i]
    else:
        pd_eurusd["open_y"].iloc[i] = pd_eurusd["open_y"].iloc[i-1]
        pd_eurusd["close_y"].iloc[i] = pd_eurusd["close_x"].iloc[i]
        if pd_eurusd["high_x"].iloc[i] > pd_eurusd["high_x"].iloc[i-1]:
            pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_x"].iloc[i]
        else:
            pd_eurusd["high_y"].iloc[i] = pd_eurusd["high_y"].iloc[i-1]
        if pd_eurusd["low_x"].iloc[i] < pd_eurusd["low_x"].iloc[i-1]:
            pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_x"].iloc[i]
        else:
            pd_eurusd["low_y"].iloc[i] = pd_eurusd["low_y"].iloc[i-1]


pd_eurusd[0:10]

,time,open_x,high_x,low_x,close_x,tick_volume_x,spread_x,real_volume_x,upperBBands_M5,middleBBands_M5,...,ADOsc_M5,OBV_M5,TrueRange_M5,open_y,high_y,low_y,close_y,tick_volume_y,spread_y,real_volume_y
3,1568768400,1.10711,1.10731,1.10706,1.10726,65,4,0,NaN,NaN,...,NaN,126.0,0.00025,1.10711,1.10731,1.10706,1.10726,252.0,3.0,0.0
4,1568768700,1.10726,1.10727,1.10717,1.10726,30,3,0,NaN,NaN,...,NaN,126.0,0.00010,1.10711,1.10731,1.10706,1.10726,NaN,NaN,NaN
5,1568769000,1.10726,1.10726,1.10720,1.10720,31,3,0,NaN,NaN,...,NaN,95.0,0.00006,1.10711,1.10731,1.10706,1.10720,NaN,NaN,NaN
6,1568769300,1.10720,1.10726,1.10715,1.10725,40,3,0,NaN,NaN,...,NaN,135.0,0.00011,1.10711,1.10731,1.10715,1.10725,NaN,NaN,NaN
7,1568769600,1.10725,1.10732,1.10721,1.10727,39,3,0,NaN,NaN,...,NaN,174.0,0.00011,1.10711,1.10732,1.10715,1.10727,NaN,NaN,NaN
8,1568769900,1.10727,1.10727,1.10716,1.10719,47,3,0,NaN,NaN,...,NaN,127.0,0.00011,1.10711,1.10732,1.10716,1.10719,NaN,NaN,NaN
9,1568770200,1.10719,1.10725,1.10716,1.10716,55,3,0,NaN,NaN,...,-8.832760,72.0,0.00009,1.10719,1.10725,1.10716,1.10716,262.0,2.0,0.0
10,1568770500,1.10716,1.10725,1.10705,1.10707,55,4,0,NaN,NaN,...,-30.979634,17.0,0.00020,1.10719,1.10725,1.10705,1.10707,NaN,NaN,NaN
11,1568770800,1.10707,1.10711,1.10705,1.10710,33,2,0,NaN,NaN,...,-30.223388,50.0,0.00006,1.10719,1.10725,1.10705,1.10710,NaN,NaN,NaN
12,1568771100,1.10710,1.10715,1.10705,1.10712,55,2,0,NaN,NaN,...,-20.166434,105.0,0.00010,1.10719,1.10715,1.10705,1.10712,NaN,NaN,NaN


In [12]:
# Adjust M30 volume

for i in range(0, len(pd_eurusd)):
    if pd_eurusd["time"].iloc[i] % 1800 == 0:
        pd_eurusd["tick_volume_y"].iloc[i] = pd_eurusd["tick_volume_x"].iloc[i]
    else:
        pd_eurusd["tick_volume_y"].iloc[i] = pd_eurusd["tick_volume_x"].iloc[i] + pd_eurusd["tick_volume_y"].iloc[i-1]

pd_eurusd[0:15]

,time,open_x,high_x,low_x,close_x,tick_volume_x,spread_x,real_volume_x,upperBBands_M5,middleBBands_M5,...,ADOsc_M5,OBV_M5,TrueRange_M5,open_y,high_y,low_y,close_y,tick_volume_y,spread_y,real_volume_y
3,1568768400,1.10711,1.10731,1.10706,1.10726,65,4,0,NaN,NaN,...,NaN,126.0,0.00025,1.10711,1.10731,1.10706,1.10726,65.0,3.0,0.0
4,1568768700,1.10726,1.10727,1.10717,1.10726,30,3,0,NaN,NaN,...,NaN,126.0,0.00010,1.10711,1.10731,1.10706,1.10726,95.0,NaN,NaN
5,1568769000,1.10726,1.10726,1.10720,1.10720,31,3,0,NaN,NaN,...,NaN,95.0,0.00006,1.10711,1.10731,1.10706,1.10720,126.0,NaN,NaN
6,1568769300,1.10720,1.10726,1.10715,1.10725,40,3,0,NaN,NaN,...,NaN,135.0,0.00011,1.10711,1.10731,1.10715,1.10725,166.0,NaN,NaN
7,1568769600,1.10725,1.10732,1.10721,1.10727,39,3,0,NaN,NaN,...,NaN,174.0,0.00011,1.10711,1.10732,1.10715,1.10727,205.0,NaN,NaN
8,1568769900,1.10727,1.10727,1.10716,1.10719,47,3,0,NaN,NaN,...,NaN,127.0,0.00011,1.10711,1.10732,1.10716,1.10719,252.0,NaN,NaN
9,1568770200,1.10719,1.10725,1.10716,1.10716,55,3,0,NaN,NaN,...,-8.832760,72.0,0.00009,1.10719,1.10725,1.10716,1.10716,55.0,2.0,0.0
10,1568770500,1.10716,1.10725,1.10705,1.10707,55,4,0,NaN,NaN,...,-30.979634,17.0,0.00020,1.10719,1.10725,1.10705,1.10707,110.0,NaN,NaN
11,1568770800,1.10707,1.10711,1.10705,1.10710,33,2,0,NaN,NaN,...,-30.223388,50.0,0.00006,1.10719,1.10725,1.10705,1.10710,143.0,NaN,NaN
12,1568771100,1.10710,1.10715,1.10705,1.10712,55,2,0,NaN,NaN,...,-20.166434,105.0,0.00010,1.10719,1.10715,1.10705,1.10712,198.0,NaN,NaN


In [13]:
# Bollinger bands M30
timeperiod = 20
nbdev = 2

count = 0
midBand = np.array([])
upBand = np.array([])
lowBand = np.array([])
tpTemp = np.array([])
#closeTemp = np.array([])


for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            tpTemp = np.append(tpTemp, tp)
            #closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            midBand = np.append(midBand, [np.nan])
            upBand = np.append(upBand, [np.nan])
            lowBand = np.append(lowBand, [np.nan])
        else:
            midBand = np.append(midBand, [np.nan])
            upBand = np.append(upBand, [np.nan])
            lowBand = np.append(lowBand, [np.nan])
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            # Calcul midBand, return midBand and new smaTemp
            # Calcul upBand, return upBand
            # Calcul lowBand, return lowBand and new closeTemp
            # Calcul midBand, return midBand
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            tpTemp = np.append(tpTemp, tp)
            smaTemp = talib.SMA(tpTemp, timeperiod= timeperiod)
            midBand = np.append(midBand, [smaTemp[-1]])
            # Calcul sigma
            std_tp = np.std(tpTemp)
            # Calcul upBand, return upBand
            temp_up = smaTemp[-1] + nbdev * std_tp
            upBand = np.append(upBand, [temp_up])
            # Calcul lowBand, return lowBand
            temp_low = smaTemp[-1] - nbdev * std_tp
            lowBand = np.append(lowBand, [temp_low])
            # Delete first tpTemp value to stay with tiemperiod-1 values
            tpTemp = np.delete(tpTemp, 0)
        else:
            # Calcul midBand, return midBand
            tp = (pd_eurusd["high_y"].iloc[i] + pd_eurusd["low_y"].iloc[i] + pd_eurusd["close_y"].iloc[i]) / 3
            temp_tp = np.append(tpTemp, tp)
            smaTemp = talib.SMA(temp_tp, timeperiod= timeperiod)
            midBand = np.append(midBand, [smaTemp[-1]])
            # Calcul sigma
            std_tp = np.std(temp_tp)
            # Calcul upBand, return upBand
            temp_up = smaTemp[-1] + nbdev * std_tp
            upBand = np.append(upBand, [temp_up])
            # Calcul lowBand, return lowBand
            temp_low = smaTemp[-1] - nbdev * std_tp
            lowBand = np.append(lowBand, [temp_low])

pd_eurusd["upperBBands_M30"] = upBand
pd_eurusd["middleBBands_M30"] = midBand
pd_eurusd["lowerBBands_M30"] = lowBand

In [14]:
# Double Exponential Moving Average ** On Close ** timeperiod = 21
timeperiod = 21

count = 0
dema = np.array([])
closeTemp = np.array([])
emaTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            dema = np.append(dema, np.nan)
        else:
            dema = np.append(dema, np.nan)
    else:
        if count < 2 * timeperiod -1:
            if pd_eurusd["time"].iloc[i] % 1800 == 1500:
                count += 1
                closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(closeTemp, timeperiod= timeperiod)
                emaTemp = np.append(emaTemp, ema1[-1])
                closeTemp = np.delete(closeTemp, 0)
                dema = np.append(dema, np.nan)
            else:
                dema = np.append(dema, np.nan)
        else:
            if pd_eurusd["time"].iloc[i] % 1800 == 1500:
                closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(closeTemp, timeperiod= timeperiod)
                emaTemp = np.append(emaTemp, ema1[-1])
                ema2 = talib.EMA(emaTemp, timeperiod= timeperiod)
                calc = 2 * ema1[-1] - ema2[-1]
                dema = np.append(dema, calc)
                closeTemp = np.delete(closeTemp, 0)
                emaTemp = np.delete(emaTemp, 0)
            else:
                c_temp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
                ema1 = talib.EMA(c_temp, timeperiod= timeperiod)
                ema_tp = np.append(emaTemp, ema1[-1])
                ema2 = talib.EMA(ema_tp, timeperiod= timeperiod)
                calc = 2 * ema1[-1] - ema2[-1]
                dema = np.append(dema, calc)

pd_eurusd["DEMA_M30"] = dema

In [25]:
# MidPoint over period * on close * timeperiod = 14
timeperiod = 14

count = 0
midpoint = np.array([])
closeTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            midpoint = np.append(midpoint, np.nan)
        else:
            midpoint = np.append(midpoint, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            closeTemp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = (closeTemp.max() + closeTemp.min()) / 2
            midpoint = np.append(midpoint, calc)
            closeTemp = np.delete(closeTemp, 0)
        else:
            c_temp = np.append(closeTemp, pd_eurusd["close_y"].iloc[i])
            calc = (c_temp.max() + c_temp.min()) / 2
            midpoint = np.append(midpoint, calc)

pd_eurusd["Midpoint_M30"] = midpoint

In [27]:
# Midpoint Price over period * on high, low * timeperiod = 14
timeperiod = 14

count = 0
midpoint = np.array([])
highTemp = np.array([])
lowTemp = np.array([])

for i in range(0, len(pd_eurusd)):
    if count < timeperiod-1:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            count += 1
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            midpoint = np.append(midpoint, np.nan)
        else:
            midpoint = np.append(midpoint, np.nan)
    else:
        if pd_eurusd["time"].iloc[i] % 1800 == 1500:
            highTemp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            lowTemp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            calc = (highTemp.max() + lowTemp.min()) / 2
            midpoint = np.append(midpoint, calc)
            highTemp = np.delete(midpoint, 0)
            lowTemp = np.delete(midpoint, 0)
        else:
            h_temp = np.append(highTemp, pd_eurusd["high_y"].iloc[i])
            l_temp = np.append(lowTemp, pd_eurusd["low_y"].iloc[i])
            calc = (h_temp.max() + l_temp.min()) / 2
            midpoint = np.append(midpoint, calc)

pd_eurusd["Midprice_M30"] = midpoint

In [28]:
pd_eurusd.describe()

,time,open_x,high_x,low_x,close_x,tick_volume_x,spread_x,real_volume_x,upperBBands_M5,middleBBands_M5,...,close_y,tick_volume_y,spread_y,real_volume_y,upperBBands_M30,middleBBands_M30,lowerBBands_M30,DEMA_M30,Midpoint_M30,Midprice_M30
count,9.999600e+04,99996.000000,99996.000000,99996.000000,99996.000000,99996.000000,99996.000000,99996.0,99980.000000,99980.000000,...,99996.000000,99996.000000,16664.000000,16664.0,99882.000000,99882.000000,99882.000000,99750.000000,99918.000000,6.000000
mean,1.590048e+09,1.137267,1.137463,1.137072,1.137268,206.818213,1.922037,0.0,1.138119,1.137262,...,1.137268,734.021421,1.062650,0.0,1.139348,1.137240,1.135132,1.137345,1.137249,1.106995
std,1.225491e+07,0.044418,0.044426,0.044410,0.044418,187.530416,3.034700,0.0,0.044446,0.044410,...,0.044418,779.118407,1.310898,0.0,0.044472,0.044375,0.044335,0.044471,0.044372,0.000000
min,1.568768e+09,1.063870,1.064400,1.063550,1.063890,1.000000,0.000000,0.0,1.066397,1.065084,...,1.063890,1.000000,0.000000,0.0,1.070249,1.067597,1.060438,1.061393,1.066555,1.106995
25%,1.579615e+09,1.101520,1.101650,1.101380,1.101520,81.000000,1.000000,0.0,1.102063,1.101476,...,1.101520,238.000000,0.000000,0.0,1.102746,1.101398,1.100125,1.101352,1.101370,1.106995
50%,1.590069e+09,1.118515,1.118760,1.118270,1.118510,155.000000,2.000000,0.0,1.119629,1.118438,...,1.118510,483.000000,1.000000,0.0,1.121092,1.118401,1.116117,1.119041,1.118302,1.106995
75%,1.600685e+09,1.180150,1.180350,1.179910,1.180143,266.000000,2.000000,0.0,1.181026,1.180148,...,1.180143,951.000000,1.000000,0.0,1.182278,1.180465,1.177790,1.180345,1.180310,1.106995
max,1.611323e+09,1.234820,1.234940,1.234480,1.234820,2677.000000,171.000000,0.0,1.235733,1.234218,...,1.234820,11144.000000,28.000000,0.0,1.236432,1.232803,1.231584,1.234345,1.233580,1.106995


In [ ]:
A1 = 0
A2 = 1

SI(A1=0, "zero", "pouet")

if A1 == 0:
    print("zero")
else:
    print("pouet")